In [1]:
import requests
from bs4 import BeautifulSoup
import re
import selenium
import time
import pandas as pd
import random
import scraping_class
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import scraping_class
from bs4 import BeautifulSoup
import re
import time

In [19]:
logfile = 'log_test.csv'
connector = scraping_class.Connector(logfile)

def get_links(): 
    # Connect to the url: 
    #response, call_id = connector.get(category_url, 'mapping_test')
    
    # Find the last page number: 
    #page_number_link_locations = response.text.split('Page number ')[1:]
    #page_numbers = []
    #for no in page_number_link_locations:
    #    no = no.split('"')[0]
    #    page_numbers.append(no)
    #last_page_no = int(page_numbers[-1])
    
    # Fetch the review links: 
    links = []  
    for i in tqdm(range(1, 1000)): 
        response, call_id = connector.get(f'https://www.dn.no/smak/vinsok?p={i}&price=0%5C%20%5C-%5C%20149', 'mapping')
        link_locations = response.text.split('href="')[1:]
        for link in link_locations:
            link = link.split('"')[0]
            links.append(link)
    links = [url+link for link in links if '/vin/' in link] 
    
    # Remove duplicates
    links = list(dict.fromkeys(links))
    
    return links

test = get_links()



 13%|█▎        | 133/999 [01:14<08:31,  1.69it/s]

KeyboardInterrupt: 

In [71]:
url = 'https://www.dn.no/smak/vinsok?p=2&price=0%5C%20%5C-%5C%20149'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
soup = BeautifulSoup(driver.page_source,'html')

table_nodes = soup.find_all('div', {'class':'vrs-article__body'})

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Caroline/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [63]:
def parse_html_tables(table_node): 
    #table_headers = table_node.thead.find_all('th')
    #table_headers = [col['aria-label'] for col in table_headers]

    table_body = table_node.tbody
    table_rows = table_body.find_all('tr')
    
    # This function extracts the information from the child and ensures the right format:
    import numpy as np
    def convert_value_text(value_node):
        if value_node.name == 'th': 
            return value_node.text
        else: # assumes node is td
            try: 
                return str(value_node.text)
            except: 
                return np.nan
    
    # This finds the children and extracts the values from all of the rows:
    row_list = []
    for row in range(len(table_rows)): 
        lst = list(table_rows[row].children)
        inner_list = []
        for col in range(len(lst)): 
            child = lst[col]
            child = convert_value_text(child)
            inner_list.append(child)
        row_list.append(inner_list)
    
    # This converts the list of lists to a dataframe:
    import pandas as pd
    #df = pd.DataFrame(row_list, columns=table_headers)
    return row_list

In [84]:
url = 'https://www.dn.no/smak/vinsok?p=2&price=0%5C%20%5C-%5C%20149'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(5)
htmlSource = driver.page_source

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Caroline/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [ ]:
links_names = []
link_locations = htmlSource.split('href="')[1:]
for link in link_locations:
    link = link.split('</a>')[0]
    links_names.append(link)
links_names = [link for link in links_names if '/vin/' in link] 
links_names = list(dict.fromkeys(links_names))
links = []
names = []
for link in links_names:
    split = link.split('>')[0]
    name = link.split('>')[1]
    links.append(split)
    names.append(name)

In [44]:
# Test samlet

def retrive_names_and_links():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    links_names = []
    links = []
    names = []

    for i in tqdm(range(1, 10)): # Change the limit of the range
        url = f'https://www.dn.no/smak/vinsok?p={i}&price=0%5C%20%5C-%5C%20149'
        driver.get(url)
        time.sleep(5)
        htmlSource = driver.page_source
        link_locations = htmlSource.split('href="')[1:]
        for link in link_locations:
            link = link.split('</a>')[0]
            links_names.append(link)
        links_names = [link for link in links_names if '/vin/' in link] 
        links_names = list(dict.fromkeys(links_names))
        for link in links_names:
            split = link.split('>')[0]
            name = link.split('>')[1]
            links.append(split)
            names.append(name)
    return links, names

retrive_names_and_links()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Caroline/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


100%|██████████| 9/9 [00:52<00:00,  5.78s/it]


[]

In [46]:
links_names = retrive_names_and_links()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Caroline/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


100%|██████████| 9/9 [00:55<00:00,  6.22s/it]


In [51]:
links = links_names[0]
names = links_names[1]

# Remove duplicates
names = list(dict.fromkeys(names))
links = list(dict.fromkeys(links))

In [52]:
df_test = pd.DataFrame([])
df_test['name'] = names
df_test['link'] = links
df_test

,name,link
0,Beer Geek Breakfast,https://www.dn.no/smak/vin/beer-geek-breakfast...
1,Boon Framboise,https://www.dn.no/smak/vin/boon-framboise/4-1-...
2,Salamandre Pur Malt,https://www.dn.no/smak/vin/salamandre-pur-malt...
3,N'Ice Chouffe,https://www.dn.no/smak/vin/nice-chouffe/4-1-30...
4,Topf Wechselberg Riesling,https://www.dn.no/smak/vin/topf-wechselberg-ri...
...,...,...
85,Prosecco Extra Dry,https://www.dn.no/smak/vin/prosecco-extra-dry/...
86,Finca Constancia Tempranillo,https://www.dn.no/smak/vin/finca-constancia-te...
87,Riondo Pink,"https://www.dn.no/smak/vin/riondo-pink/4-1-25907"""
88,DIVICI Organic Prosecco,https://www.dn.no/smak/vin/divici-organic-pros...


In [61]:
# Test search names on Vivino
search_name = []
for name in names: 
    search_name.append(str(name).replace(' ', '+'))
    
driver = webdriver.Chrome(ChromeDriverManager().install())

vivino_link_locations = []
vivino_links_names = []
vivino_links = []
vivino_names = []

for name in tqdm(search_name[0:2]): 
    search_url = f'https://www.vivino.com/search/wines?q={name}'
    driver.get(search_url)
    time.sleep(5)
    htmlSource = driver.page_source
    vivino_link_locations = htmlSource.split('href="')[1:]
    for link in vivino_link_locations:
        link = link.split('</a>')[0]
        vivino_links_names.append(link)
    vivino_links_names = [link for link in vivino_links_names if '"@id":' in link] 
    vivino_links_names = list(dict.fromkeys(vivino_links_names))
    for link in vivino_links_names:
        split = link.split('"@id":')
        print(split)
#        name = link.split('"')[2]
#        vivino_links.append(split)
#        vivino_names.append(name)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/Caroline/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


 50%|█████     | 1/2 [00:07<00:07,  7.56s/it]

['/assets/application-42927b43222e908ef52255f0377581b0d311bd29f4470cf8069b6c1f5ae57acc.css" data-turbolinks-track="true">\n<!-- HTML5 shim and Respond.js IE8 support of HTML5 elements and media queries -->\n<!--[if lt IE 9]>\n<script src="https://oss.maxcdn.com/libs/respond.js/1.3.0/respond.min.js"></script>\n<![endif]-->\n<!-- Google Analytics tracking code. -->\n<script src="https://connect.facebook.net/signals/config/564683513994910?v=2.9.23&amp;r=stable" async=""></script><script src="https://connect.facebook.net/en_US/sdk.js?hash=0fa5a71ff5daba58e577d5bf97f6410f&amp;ua=modern_es6" async="" crossorigin="anonymous"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script type="text/javascript" defer="" src="https://d31qbv1cthcecs.cloudfront.net/atrk.js"></script><script type="text/javascript" async="" src="https://cdn.heapanalytics.com/js/heap-3503103446.js"></script><script async="" src="//www.google-analytics.com/analytics.js"></script><scrip

100%|██████████| 2/2 [00:13<00:00,  6.96s/it]

['/assets/application-42927b43222e908ef52255f0377581b0d311bd29f4470cf8069b6c1f5ae57acc.css" data-turbolinks-track="true">\n<!-- HTML5 shim and Respond.js IE8 support of HTML5 elements and media queries -->\n<!--[if lt IE 9]>\n<script src="https://oss.maxcdn.com/libs/respond.js/1.3.0/respond.min.js"></script>\n<![endif]-->\n<!-- Google Analytics tracking code. -->\n<script src="https://connect.facebook.net/signals/config/564683513994910?v=2.9.23&amp;r=stable" async=""></script><script src="https://connect.facebook.net/en_US/sdk.js?hash=0fa5a71ff5daba58e577d5bf97f6410f&amp;ua=modern_es6" async="" crossorigin="anonymous"></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script type="text/javascript" defer="" src="https://d31qbv1cthcecs.cloudfront.net/atrk.js"></script><script type="text/javascript" async="" src="https://cdn.heapanalytics.com/js/heap-3503103446.js"></script><script async="" src="//www.google-analytics.com/analytics.js"></script><scrip

In [70]:
#display(vivino_links)
#display(vivino_names)
splitted = [link.split('"@id":') for link in vivino_links_names]
splitted = [link.split('"name":') for link in splitted]


AttributeError: 'list' object has no attribute 'split'

In [ ]:
# step 1: load the webpage we want to scrape in our virtual browser
url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)




In [ ]:
# step 2: scroll down the page to load more profiles
import time

url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(3)

lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(1)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

In [ ]:
# step 3: save the soup and keep track of runtime

import time
from bs4 import BeautifulSoup

start_time = time.time()

url = 'https://nboard.dk/search'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

time.sleep(3)

lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(1)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'lxml')

print("--- %s seconds ---" % round((time.time() - start_time),2))